In [1]:
!pip install -q transformers torchaudio soundfile

from transformers import WhisperProcessor, WhisperForConditionalGeneration
import torch
import soundfile as sf
import os

model_id = "openai/whisper-large-v3"
processor = WhisperProcessor.from_pretrained(model_id)
model = WhisperForConditionalGeneration.from_pretrained(model_id)
device = "cuda" if torch.cuda.is_available() else "cpu"
model.to(device)


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 52.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 35.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 21.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 7.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 14.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 8.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 4.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 24.7 MB/s eta 0:00:00


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


preprocessor_config.json:   0%|          | 0.00/340 [00:00<?, ?B/s]

tokenizer_config.json: 0.00B [00:00, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

...

special_tokens_map.json: 0.00B [00:00, ?B/s]

config.json: 0.00B [00:00, ?B/s]

model.safetensors:   0%|          | 0.00/3.09G [00:00<?, ?B/s]

generation_config.json: 0.00B [00:00, ?B/s]

WhisperForConditionalGeneration(
  (model): WhisperModel(
    (encoder): WhisperEncoder(
      (conv1): Conv1d(128, 1280, kernel_size=(3,), stride=(1,), padding=(1,))
      (conv2): Conv1d(1280, 1280, kernel_size=(3,), stride=(2,), padding=(1,))
      (embed_positions): Embedding(1500, 1280)
      (layers): ModuleList(
        (0-31): 32 x WhisperEncoderLayer(
          (self_attn): WhisperAttention(
            (k_proj): Linear(in_features=1280, out_features=1280, bias=False)
            (v_proj): Linear(in_features=1280, out_features=1280, bias=True)
            (q_proj): Linear(in_features=1280, out_features=1280, bias=True)
            (out_proj): Linear(in_features=1280, out_features=1280, bias=True)
          )
          (self_attn_layer_norm): LayerNorm((1280,), eps=1e-05, elementwise_affine=True)
          (activation_fn): GELUActivation()
          (fc1): Linear(in_features=1280, out_features=5120, bias=True)
          (fc2): Linear(in_features=5120, out_features=1280, bias=Tr

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
import tarfile
import os

input_tar_gz = "/content/drive/MyDrive/cv-corpus-22.0-delta-2025-06-20-en.tar.gz"
extract_dir = "/tmp/extracted_audio"

os.makedirs(extract_dir, exist_ok=True)

with tarfile.open(input_tar_gz, "r:gz") as tar:
    tar.extractall(path=extract_dir)

print(f"Extracted contents to: {extract_dir}")

Extracted contents to: /tmp/extracted_audio


In [4]:
import pandas as pd
import os

extract_dir = "/tmp/extracted_audio"

# List files in the extracted directory to identify the manifest file
extracted_files = os.listdir(extract_dir)
print("Files in extracted directory:", extracted_files)

# Assuming the manifest file is a TSV named "validated.tsv" based on common Common Voice datasets
manifest_file_path = os.path.join(extract_dir, "en", "validated.tsv")

# Read the manifest file into a pandas DataFrame
try:
    metadata_df = pd.read_csv(manifest_file_path, sep='\t')
    print("\nMetadata DataFrame:")
    display(metadata_df.head())
except FileNotFoundError:
    print(f"Manifest file not found at: {manifest_file_path}")
    metadata_df = None
except Exception as e:
    print(f"Error reading or parsing the manifest file: {e}")
    metadata_df = None


Files in extracted directory: ['cv-corpus-22.0-delta-2025-06-20']
Manifest file not found at: /tmp/extracted_audio/en/validated.tsv


In [5]:
import pandas as pd
import os

extract_dir = "/tmp/extracted_audio"

# List files in the nested directory to find the manifest file
nested_dir = os.path.join(extract_dir, "cv-corpus-22.0-delta-2025-06-20")
nested_files = os.listdir(nested_dir)
print("Files in nested directory:", nested_files)

# Assuming the manifest file is now in the nested directory and named "validated.tsv"
manifest_file_path = os.path.join(nested_dir, "en", "validated.tsv")

# Read the manifest file into a pandas DataFrame
try:
    metadata_df = pd.read_csv(manifest_file_path, sep='\t')
    print("\nMetadata DataFrame:")
    display(metadata_df.head())
except FileNotFoundError:
    print(f"Manifest file not found at: {manifest_file_path}")
    metadata_df = None
except Exception as e:
    print(f"Error reading or parsing the manifest file: {e}")
    metadata_df = None

Files in nested directory: ['en']

Metadata DataFrame:


,client_id,path,sentence_id,sentence,sentence_domain,up_votes,down_votes,age,gender,accents,variant,locale,segment
0,02c9eef1805cc4b7c5922f0b816def65ae4c4997f1c332...,common_voice_en_43199993.mp3,fa7ef4a14583f9cd09429471d8740edb235f07edf30580...,"In this phase, the party was based in Eastern ...",NaN,2,0,NaN,NaN,"British English,Cornish English",NaN,en,NaN
1,06c3ce6c362cb98ab3d50d2b2c34bdd84e73584e9a7a5a...,common_voice_en_42736613.mp3,f8a8adc4e17873eca165691ef65cc4fbf61f02e86b8958...,There is also an interchange with the Thousand...,NaN,2,0,twenties,female_feminine,United States English,NaN,en,NaN
2,0f96302cb9d236c534f6608f1b152eeba8ea295b975d82...,common_voice_en_42798328.mp3,f941273f163f3cd7e1d91f29f6f3d9f7d7e8b34b9885d4...,"Five days later, Royal Marines boarded the pla...",NaN,2,0,thirties,female_feminine,England English,NaN,en,NaN
3,134e0fd7b344e38bd25fc9ba8acf1372406f2a1099378c...,common_voice_en_43204215.mp3,fb31892179330d3e63a25d50a63b9ef2535cd29192b041...,Only a small Greek state became independent in...,NaN,2,0,NaN,NaN,"United States English,United States English, b...",NaN,en,NaN
4,15e07adfee8c774664386125e1b22ed62ade8b32e67204...,common_voice_en_42706055.mp3,f8986faeab37c4e298d55baaef72dcf38692974d82bb1d...,Knob Noster State Park is nearby.,NaN,2,0,fourties,NaN,"India and South Asia (India, Pakistan, Sri Lanka)",NaN,en,NaN


**Reasoning**:
Iterate through the metadata DataFrame, create folders based on gender and age, and move audio files to the respective folders.



In [6]:
import os
import shutil

# Define base directories
extracted_audio_base_dir = "/tmp/extracted_audio/cv-corpus-22.0-delta-2025-06-20/en/clips"
organized_audio_base_dir = "/tmp/organized_audio"

# Create the base directory for organized audio files
os.makedirs(organized_audio_base_dir, exist_ok=True)

# Iterate through each row of the metadata_df DataFrame
for index, row in metadata_df.iterrows():
    # Extract metadata for folder creation
    gender = row.get('gender', 'unknown') # Use 'unknown' for missing gender
    age = row.get('age', 'unknown') # Use 'unknown' for missing age
    audio_file_name = row['path']

    # Construct the target folder path
    target_folder = os.path.join(organized_audio_base_dir, f"gender_{gender}", f"age_{age}")
    os.makedirs(target_folder, exist_ok=True)

    # Construct source and destination paths
    source_path = os.path.join(extracted_audio_base_dir, audio_file_name)
    destination_path = os.path.join(target_folder, audio_file_name)

    # Move the audio file
    try:
        shutil.move(source_path, destination_path)
    except FileNotFoundError:
        print(f"Warning: Source file not found at {source_path}. Skipping.")
    except Exception as e:
        print(f"Error moving file {source_path} to {destination_path}: {e}")

print("Audio files organized successfully.")

Audio files organized successfully.


In [7]:
import shutil
import os

# Define the source directory where the organized audio files are located
source_dir = "/tmp/organized_audio"

# Define the destination directory in your Google Drive
destination_dir = "/content/drive/MyDrive/organized_common_voice_audio"

# Create the destination directory if it doesn't exist
os.makedirs(destination_dir, exist_ok=True)

# Use shutil.copytree() to recursively copy the directory
try:
    shutil.copytree(source_dir, destination_dir, dirs_exist_ok=True)
    print(f"Successfully copied organized audio files from {source_dir} to {destination_dir}")
except FileNotFoundError:
    print(f"Error: Source directory not found at {source_dir}")
except FileExistsError:
    print(f"Error: Destination directory {destination_dir} already exists and cannot be overwritten directly with copytree unless dirs_exist_ok=True is used.")
except Exception as e:
    print(f"An error occurred during copying: {e}")


Successfully copied organized audio files from /tmp/organized_audio to /content/drive/MyDrive/organized_common_voice_audio


In [8]:
import shutil
import os

# Define the path to the temporary directory
extract_dir = "/tmp/extracted_audio"

# Check if the directory exists before removing it
if os.path.exists(extract_dir):
    # Remove the temporary directory and all its contents
    shutil.rmtree(extract_dir)
    print(f"Temporary extracted files removed from: {extract_dir}")
else:
    print(f"Temporary extracted directory not found: {extract_dir}")


Temporary extracted files removed from: /tmp/extracted_audio


In [9]:
import os

# Define the root directory of the organized audio files in Google Drive
organized_audio_root_dir = "/content/drive/MyDrive/organized_common_voice_audio"

# List to store the paths of all audio files
audio_file_paths = []

# Walk through the directory structure
for root, _, files in os.walk(organized_audio_root_dir):
    for file in files:
        # Check if the file is an MP3 audio file (assuming all are .mp3)
        if file.endswith(".mp3"):
            audio_file_paths.append(os.path.join(root, file))

# Print the number of found audio files and the first few paths
print(f"Found {len(audio_file_paths)} audio files.")
print("First 5 audio file paths:", audio_file_paths[:5])

Found 170 audio files.
First 5 audio file paths: ['/content/drive/MyDrive/organized_common_voice_audio/gender_nan/age_nan/common_voice_en_43199122.mp3', '/content/drive/MyDrive/organized_common_voice_audio/gender_nan/age_nan/common_voice_en_43375391.mp3', '/content/drive/MyDrive/organized_common_voice_audio/gender_nan/age_nan/common_voice_en_43203141.mp3', '/content/drive/MyDrive/organized_common_voice_audio/gender_nan/age_nan/common_voice_en_43196248.mp3', '/content/drive/MyDrive/organized_common_voice_audio/gender_nan/age_nan/common_voice_en_42814135.mp3']


In [18]:
import torchaudio

# Define the root directory for saving the transcriptions in Google Drive
transcriptions_root_dir = "/content/drive/MyDrive/audio_transcriptions"

# Create the root directory for transcriptions if it doesn't exist
os.makedirs(transcriptions_root_dir, exist_ok=True)

# Iterate through each audio file path
for audio_file_path in audio_file_paths:
    try:
        # Construct the corresponding transcription file path
        relative_path = os.path.relpath(audio_file_path, organized_audio_root_dir)
        transcription_file_path = os.path.join(transcriptions_root_dir, os.path.splitext(relative_path)[0] + ".txt")

        # Create the directory for the transcription file if it doesn't exist
        os.makedirs(os.path.dirname(transcription_file_path), exist_ok=True)

        # Load and resample the audio file using torchaudio
        audio, sr = torchaudio.load(audio_file_path)

        # Resample if the sampling rate is not 16000 Hz
        if sr != 16000:
            resampler = torchaudio.transforms.Resample(orig_freq=sr, new_freq=16000)
            audio = resampler(audio)
            sr = 16000 # Update sampling rate

        # Ensure the audio tensor is 1D (mono) and on the correct device
        if audio.ndim > 1:
            audio = audio.mean(dim=0) # Average channels to make it mono

        # Ensure the audio tensor is float32 as expected by the processor
        audio = audio.to(torch.float32).squeeze()


        inputs = processor(audio, sampling_rate=sr, return_tensors="pt").to(device)
        predicted_ids = model.generate(inputs.input_features)
        transcription = processor.batch_decode(predicted_ids, skip_special_tokens=True)[0]

        # Save the transcription to a text file
        with open(transcription_file_path, "w") as f:
            f.write(transcription)

        print(f"Transcribed '{audio_file_path}' and saved to '{transcription_file_path}'")

    except FileNotFoundError:
        print(f"Error: Audio file not found at '{audio_file_path}'. Skipping.")
    except Exception as e:
        print(f"Error processing file '{audio_file_path}': {e}")

print("\nTranscription process completed.")

Using custom `forced_decoder_ids` from the (generation) config. This is deprecated in favor of the `task` and `language` flags/config options.
Transcription using a multilingual Whisper will default to language detection followed by transcription instead of translation to English. This might be a breaking change for your use case. If you want to instead always translate your audio to English, make sure to pass `language='en'`. See https://github.com/huggingface/transformers/pull/28687 for more details.
The attention mask is not set and cannot be inferred from input because pad token is same as eos token. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.


Transcribed '/content/drive/MyDrive/organized_common_voice_audio/gender_nan/age_nan/common_voice_en_43199122.mp3' and saved to '/content/drive/MyDrive/audio_transcriptions/gender_nan/age_nan/common_voice_en_43199122.txt'
Transcribed '/content/drive/MyDrive/organized_common_voice_audio/gender_nan/age_nan/common_voice_en_43375391.mp3' and saved to '/content/drive/MyDrive/audio_transcriptions/gender_nan/age_nan/common_voice_en_43375391.txt'
Transcribed '/content/drive/MyDrive/organized_common_voice_audio/gender_nan/age_nan/common_voice_en_43203141.mp3' and saved to '/content/drive/MyDrive/audio_transcriptions/gender_nan/age_nan/common_voice_en_43203141.txt'
Transcribed '/content/drive/MyDrive/organized_common_voice_audio/gender_nan/age_nan/common_voice_en_43196248.mp3' and saved to '/content/drive/MyDrive/audio_transcriptions/gender_nan/age_nan/common_voice_en_43196248.txt'
Transcribed '/content/drive/MyDrive/organized_common_voice_audio/gender_nan/age_nan/common_voice_en_42814135.mp3' an

In [26]:
audio = audio.to(torch.float32).squeeze()

# Use the processor to prepare the audio data for the model
inputs = processor(audio, sampling_rate=sr, return_tensors="pt").to(device)

In [25]:
predicted_ids = model.generate(inputs.input_features)

In [24]:
transcription = processor.batch_decode(predicted_ids, skip_special_tokens=True)[0]

In [22]:
# Construct the corresponding transcription file path
relative_path = os.path.relpath(audio_file_path, organized_audio_root_dir)
transcription_file_path = os.path.join(transcriptions_root_dir, os.path.splitext(relative_path)[0] + ".txt")

# Create the directory for the transcription file if it doesn't exist
os.makedirs(os.path.dirname(transcription_file_path), exist_ok=True)

# Save the transcription to a text file
with open(transcription_file_path, "w") as f:
    f.write(transcription)

print(f"Transcribed '{audio_file_path}' and saved to '{transcription_file_path}'")

Transcribed '/content/drive/MyDrive/organized_common_voice_audio/gender_female_feminine/age_thirties/common_voice_en_42817051.mp3' and saved to '/content/drive/MyDrive/audio_transcriptions/gender_female_feminine/age_thirties/common_voice_en_42817051.txt'


In [23]:
import torchaudio
import os

# Define the root directory for saving the transcriptions in Google Drive
transcriptions_root_dir = "/content/drive/MyDrive/audio_transcriptions"

# Create the root directory for transcriptions if it doesn't exist
os.makedirs(transcriptions_root_dir, exist_ok=True)

# Iterate through each audio file path
for audio_file_path in audio_file_paths:
    try:
        # Construct the corresponding transcription file path
        relative_path = os.path.relpath(audio_file_path, organized_audio_root_dir)
        transcription_file_path = os.path.join(transcriptions_root_dir, os.path.splitext(relative_path)[0] + ".txt")

        # Create the directory for the transcription file if it doesn't exist
        os.makedirs(os.path.dirname(transcription_file_path), exist_ok=True)

        # Load and resample the audio file using torchaudio
        try:
            audio, sr = torchaudio.load(audio_file_path)
        except FileNotFoundError:
            print(f"Error loading file: Audio file not found at '{audio_file_path}'. Skipping.")
            continue # Skip to the next file
        except Exception as e:
            print(f"Error loading file '{audio_file_path}': {e}. Skipping.")
            continue # Skip to the next file

        # Resample if the sampling rate is not 16000 Hz
        if sr != 16000:
            try:
                resampler = torchaudio.transforms.Resample(orig_freq=sr, new_freq=16000)
                audio = resampler(audio)
                sr = 16000 # Update sampling rate
            except Exception as e:
                print(f"Error resampling file '{audio_file_path}': {e}. Skipping.")
                continue # Skip to the next file

        # Ensure the audio tensor is 1D (mono) and on the correct device
        if audio.ndim > 1:
            audio = audio.mean(dim=0) # Average channels to make it mono

        # Ensure the audio tensor is float32 as expected by the processor
        audio = audio.to(torch.float32).squeeze()

        # Use the processor to prepare the audio data for the model
        try:
            inputs = processor(audio, sampling_rate=sr, return_tensors="pt").to(device)
        except Exception as e:
            print(f"Error processing audio for file '{audio_file_path}': {e}. Skipping.")
            continue # Skip to the next file


        # Use the loaded model to generate the predicted token IDs from the input features
        try:
            predicted_ids = model.generate(inputs.input_features)
        except RuntimeError as e:
             print(f"Error during model generation for file '{audio_file_path}': {e}. This might be a CUDA/GPU issue. Skipping.")
             continue # Skip to the next file
        except Exception as e:
            print(f"Error during model generation for file '{audio_file_path}': {e}. Skipping.")
            continue # Skip to the next file


        # Decode the predicted token IDs back into a human-readable transcription string
        try:
            transcription = processor.batch_decode(predicted_ids, skip_special_tokens=True)[0]
        except Exception as e:
            print(f"Error decoding transcription for file '{audio_file_path}': {e}. Skipping.")
            continue # Skip to the next file


        # Save the transcription to a text file
        try:
            with open(transcription_file_path, "w") as f:
                f.write(transcription)
            print(f"Transcribed '{audio_file_path}' and saved to '{transcription_file_path}'")
        except Exception as e:
            print(f"Error saving transcription for file '{audio_file_path}': {e}. Skipping.")
            continue # Skip to the next file

    except Exception as e:
        # Catch any other unexpected errors during the process for a file
        print(f"An unexpected error occurred while processing file '{audio_file_path}': {e}. Skipping.")
        continue # Skip to the next file


print("\nTranscription process completed.")

Transcribed '/content/drive/MyDrive/organized_common_voice_audio/gender_nan/age_nan/common_voice_en_43199122.mp3' and saved to '/content/drive/MyDrive/audio_transcriptions/gender_nan/age_nan/common_voice_en_43199122.txt'
Transcribed '/content/drive/MyDrive/organized_common_voice_audio/gender_nan/age_nan/common_voice_en_43375391.mp3' and saved to '/content/drive/MyDrive/audio_transcriptions/gender_nan/age_nan/common_voice_en_43375391.txt'
Transcribed '/content/drive/MyDrive/organized_common_voice_audio/gender_nan/age_nan/common_voice_en_43203141.mp3' and saved to '/content/drive/MyDrive/audio_transcriptions/gender_nan/age_nan/common_voice_en_43203141.txt'
Transcribed '/content/drive/MyDrive/organized_common_voice_audio/gender_nan/age_nan/common_voice_en_43196248.mp3' and saved to '/content/drive/MyDrive/audio_transcriptions/gender_nan/age_nan/common_voice_en_43196248.txt'
Transcribed '/content/drive/MyDrive/organized_common_voice_audio/gender_nan/age_nan/common_voice_en_42814135.mp3' an

In [28]:
from tqdm import tqdm
import torchaudio
import os
import soundfile as sf

# Define the root directory of the organized audio files in Google Drive
organized_audio_root_dir = "/content/drive/MyDrive/organized_common_voice_audio"

# List to store the paths of all audio files and their references
samples = []

# Walk through the directory structure to get audio file paths and references
for root, _, files in os.walk(organized_audio_root_dir):
    for file in files:
        # Check if the file is an MP3 audio file
        if file.endswith(".mp3"):
            audio_path = os.path.join(root, file)
            # Find the corresponding sentence in the metadata_df
            # This assumes that the filename (without extension) is in the 'path' column of metadata_df
            file_name_without_ext = os.path.splitext(file)[0]
            matching_row = metadata_df[metadata_df['path'].str.startswith(file_name_without_ext)]

            if not matching_row.empty:
                reference = matching_row.iloc[0]['sentence']
                samples.append((audio_path, reference))
            else:
                print(f"Warning: No matching metadata found for file: {file}. Skipping.")


results = []
# Iterate through each audio file path and its reference
for audio_path, reference in tqdm(samples):
    try:
        # Load and resample the audio file using torchaudio
        audio, sr = torchaudio.load(audio_path)

        # Resample if the sampling rate is not 16000 Hz
        if sr != 16000:
            resampler = torchaudio.transforms.Resample(orig_freq=sr, new_freq=16000)
            audio = resampler(audio)
            sr = 16000 # Update sampling rate

        # Ensure the audio tensor is 1D (mono) and on the correct device
        if audio.ndim > 1:
            audio = audio.mean(dim=0) # Average channels to make it mono

        # Ensure the audio tensor is float32 as expected by the processor
        audio = audio.to(torch.float32).squeeze()

        # Use the processor to prepare the audio data for the model
        inputs = processor(audio, sampling_rate=sr, return_tensors="pt").to(device)

        # Use the loaded model to generate the predicted token IDs from the input features
        predicted_ids = model.generate(inputs.input_features)

        # Decode the predicted token IDs back into a human-readable transcription string
        transcription = processor.batch_decode(predicted_ids, skip_special_tokens=True)[0]

        results.append({"file": audio_path, "reference": reference, "prediction": transcription})

    except FileNotFoundError:
        print(f"Error: Audio file not found at '{audio_file_path}'. Skipping.")
    except Exception as e:
        print(f"Error processing file '{audio_file_path}': {e}. Skipping.")

print("\nTranscription process completed.")

100%|██████████| 170/170 [07:56<00:00,  2.80s/it]


Transcription process completed.


In [34]:
!pip install -q datasets==2.14.6 jiwer==2.3.0

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.5/50.5 kB 3.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 493.7/493.7 kB 28.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 166.4/166.4 kB 16.0 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2025.3.2 requires fsspec==2025.3.2, but you have fsspec 2023.10.0 which is incompatible.


In [38]:
from jiwer import wer

references = [r["reference"] for r in results]
predictions = [r["prediction"] for r in results]

# Calculate WER using jiwer.wer directly
error_rate = wer(references, predictions)

print(f" WER: {error_rate:.2%}")

 WER: 41.20%
